<a href="https://colab.research.google.com/github/JkoolCode/TFM_Maestria_IA/blob/main/001_Preprocesamiento_Continuacion_EDA_Options_SPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lectura CSV

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_spy_2020 = pd.read_csv('spy_options_call_output_2020.csv')
df_spy_2021 = pd.read_csv('spy_options_call_output_2021.csv')
df_spy_2022 = pd.read_csv('spy_options_call_output_2022.csv')
df_spy_2023 = pd.read_csv('spy_options_call_output_2023.csv')

In [ ]:
df_spy = pd.concat([df_spy_2020, df_spy_2021, df_spy_2022, df_spy_2023])

In [ ]:
cant_reg_inicial = df_spy.shape[0]
cant_reg_inicial

315305

In [ ]:
# Ordenar cronológicamente el dataset
df_spy = df_spy.sort_values(by=['quote_date', 'expire_date'], ascending=True).reset_index(drop=True)

In [ ]:
# Verificar si esta ordenado correctamente
print(df_spy['quote_date'].head())
print(df_spy['quote_date'].tail())

0    2020-01-02
1    2020-01-02
2    2020-01-02
3    2020-01-02
4    2020-01-02
Name: quote_date, dtype: object
315300    2023-12-29
315301    2023-12-29
315302    2023-12-29
315303    2023-12-29
315304    2023-12-29
Name: quote_date, dtype: object


## Asegurar formatos

In [ ]:
df_spy['quote_date'] = pd.to_datetime(df_spy['quote_date'])

In [ ]:
df_spy['expire_date'] = pd.to_datetime(df_spy['expire_date'])

## Nuevas Variables

In [ ]:
# Incluir anio
df_spy['anio'] = df_spy['quote_date'].dt.year

In [ ]:
df_spy['spread'] = (df_spy['c_ask'] - df_spy['c_bid'])

In [ ]:
df_spy['relative_spread'] = (df_spy['c_ask'] - df_spy['c_bid']) / df_spy['c_mid_price']

In [ ]:
df_spy['dte_year'] = df_spy['dte'] / 252 # Days To Expire in a year. La fórmula Black Scholes (BS para determinar el precio teorico de una opcion) usa T en años.

In [ ]:
df_spy['c_volume_log'] = np.log1p(df_spy['c_volume']) # Esto reduce el enorme sesgo entre opciones con volumen 0, 1, 10, 1000, etc.

# Normalización de c_volume_log por día
df_spy['c_volume_log_scaled'] = df_spy.groupby('quote_date')['c_volume_log'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-9)
)

In [ ]:
df_spy['volume_zero'] = (df_spy['c_volume'] == 0) # Esto ayuda a evitar que operaciones muy ilíquidas distorsionen el modelo.
df_spy['volume_zero'] = df_spy['volume_zero'].astype(int)

In [ ]:
# Moneyness = relación entre S (underlying_last) y K (strike), que determina si la opción es ITM, ATM o OTM.
df_spy['moneyness'] = df_spy['underlying_last'] / df_spy['strike'] # IF moneyness > 1, ITM; IF moneyness < 0, OTM; ELSE, ATM

In [ ]:
# iv_pct_rank: Permite al modelo entender si esa strike tiene IV alta dentro del día
df_spy['iv_pct_rank'] = df_spy.groupby('quote_date')['c_iv'].rank(pct=True) # siestás comprando caro o barato frente al resto del chain,

In [ ]:
# c_iv_min: identifica el c_iv minimo por dia (la volatilidad minima del dia)
c_iv_min = df_spy.loc[df_spy.groupby('quote_date')['strike_distance'].idxmin(), ['quote_date','c_iv']]
c_iv_min = c_iv_min.rename(columns={'c_iv':'c_iv_min'})

df_spy = df_spy.merge(c_iv_min, on='quote_date')

In [ ]:
# iv_skew (sesgo entre c_iv - c_iv_min) es muy importante en vencimientos cortos
df_spy['iv_skew'] = df_spy['c_iv'] - df_spy['c_iv_min']

In [ ]:
# El Spread Relativo es esencial para saber qué tan confiable es el precio last.
df_spy['spread_mid_price'] = (df_spy['c_ask'] - df_spy['c_bid']) / df_spy['c_mid_price']

## Eliminar Outliers

In [ ]:
# Eliminar c_mid_price menores o iguales a 0.005 (contratos sin liquidez)
opciones_iliquidas = df_spy[(df_spy['c_mid_price'] <= 0.005)].shape[0]
print(f"% opciones iliquidas X: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

# Eliminar outliers
df_spy = df_spy[~((df_spy['c_mid_price'] <= 0.005))]

% opciones iliquidas X: 32.97378728532691% -> (103968)


In [ ]:
# Eliminar opciones sin compradores (bid), sin vendedores (ask) y sin volumen. Opciones sin liquidez (contratos sin mercado real).
opciones_iliquidas = df_spy[(df_spy['c_bid'] == 0) & (df_spy['c_ask'] == 0) & (df_spy['c_volume'] == 0)].shape[0]
print(f"% opciones iliquidas 1: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

# Eliminar outliers
df_spy = df_spy[~((df_spy['c_bid'] == 0) & (df_spy['c_ask'] == 0) & (df_spy['c_volume'] == 0))]

% opciones iliquidas 1: 0.0% -> (0)


In [ ]:
# c_last = 0, indica el precio de contratos sin transacciones recientes (opciones sin liquidez, no hubo traders en esa opcion en ese momento, la opcion se quedo completamente OTM)
# Esto ocurre cuando los contratos estan muy lejos del dinero, con DTE corto y volumen cero.
opciones_iliquidas = df_spy[(df_spy['c_last'] == 0) & (df_spy['c_volume'] == 0)].shape[0]
print(f"% opciones iliquidas 2: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

# Eliminar outliers
df_spy = df_spy[~((df_spy['c_last'] == 0) & (df_spy['c_volume'] == 0))]

% opciones iliquidas 2: 0.01182944775406105% -> (25)


**Criterios de mercado (Resumen):**
- bid = 0, ask = 0, volume = 0 (contratos sin mercado real)
- bid = 0, ask > 0, volume = 0 (hay vendedores, pero no compradores)
- bid > 0, ask = 0, volume = 0 (hay compradores, pero no vendedores)
- bid > 0, ask > 0, volume = 0 (hay compradores y vendedores sin ejecutar una operacion)

In [ ]:
# Eliminar volatilidad implicita por criterio experto (basado en analisis de los datos)
cant_contratos_prev = df_spy.shape[0]
# Eliminar outliers
df_spy = df_spy[(df_spy['c_iv'] > 0.01) & (df_spy['c_iv'] < 120)] # c_iv que superan estos rangos, son contratos sin liquidez.

opciones_iliquidas = cant_contratos_prev - df_spy.shape[0]
print(f"% opciones iliquidas 3: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

% opciones iliquidas 3: 5.480901703172733% -> (10980)


In [ ]:
# Como nuestro objetivo es predecir c_last (el precio de la opcion), eliminaremos el volumen igual a 0, ya que no aporta a la predictibilidad del modelo.
# Si el volumen es igual a 0, significa que el contrato asociado es irrelevante y no tiene liquidez. El modelo debe aprender a predecir el precio de contratos relevantes con liquidez.
opciones_iliquidas_4 = df_spy[(df_spy['c_volume'] == 0)].shape[0]
print(f"% opciones iliquidas 4: {(opciones_iliquidas_4 / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas_4})")

# Eliminar outliers
df_spy = df_spy[~((df_spy['c_volume'] == 0))]

% opciones iliquidas 4: 20.769023421120938% -> (41607)


In [ ]:
# Eliminar c_theta menores a -1
cant_contratos_prev = df_spy.shape[0]
# Eliminar outliers
df_spy = df_spy[(df_spy['c_theta'] > -1)] # (c_theta debe estar entre –1 y 0)

opciones_iliquidas = cant_contratos_prev - df_spy.shape[0]
print(f"% opciones iliquidas 5: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

% opciones iliquidas 5: 0.4023050307105491% -> (636)


In [ ]:
# Eliminar c_gamma menores a 0
opciones_iliquidas = df_spy[(df_spy['c_gamma'] < 0)].shape[0]
print(f"% opciones iliquidas 6: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

# Eliminar outliers
df_spy = df_spy[~((df_spy['c_gamma'] < 0))]

% opciones iliquidas 6: 0.1454876683387206% -> (230)


In [ ]:
# Eliminar c_vega menores a 0
opciones_iliquidas = df_spy[(df_spy['c_vega'] < 0)].shape[0]
print(f"% opciones iliquidas 7: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

# Eliminar outliers
df_spy = df_spy[~((df_spy['c_vega'] < 0))]

% opciones iliquidas 7: 0.01710387117617621% -> (27)


In [ ]:
# Eliminar moneyness inferiores a 0.5
opciones_iliquidas = df_spy[(df_spy['moneyness'] < 0.5)].shape[0]
print(f"% opciones iliquidas 8: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

# Eliminar outliers
df_spy = df_spy[~((df_spy['moneyness'] < 0.5))]

% opciones iliquidas 8: 0.0% -> (0)


In [ ]:
# Eliminar iv_skew mayor a 10 (sesgos en la volatilidad inducidas por contratos sin liquidez)
opciones_iliquidas = df_spy[(df_spy['iv_skew'] > 10)].shape[0]
print(f"% opciones iliquidas 9: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

# Eliminar outliers
df_spy = df_spy[~((df_spy['iv_skew'] > 10))]

% opciones iliquidas 9: 0.0918698362816159% -> (145)


In [ ]:
# Eliminar c_iv mayor a 300% (estan muy OTM)
cant_contratos_prev = df_spy.shape[0]
# Eliminar outliers
df_spy = df_spy[(df_spy['c_iv'] < 3)]

opciones_iliquidas = cant_contratos_prev - df_spy.shape[0]
print(f"% opciones iliquidas 10: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

% opciones iliquidas 10: 1.5965569006951916% -> (2478)


In [ ]:
# Eliminar spread_mid_price inferiores a 0
opciones_iliquidas = df_spy[(df_spy['spread_mid_price'] < 0)].shape[0]
print(f"% opciones iliquidas 11: {(opciones_iliquidas / df_spy.shape[0]) * 100}% -> ({opciones_iliquidas})")

# Eliminar outliers
df_spy = df_spy[~((df_spy['spread_mid_price'] < 0))]

% opciones iliquidas 11: 0.23709965272632386% -> (368)


In [ ]:
cant_reg_final = df_spy.shape[0]
cant_reg_final

154841

In [ ]:
print(f"Cantidad de registros Inicial: {cant_reg_inicial}")
diff = cant_reg_inicial - cant_reg_final
print(f"% de reduccion de registros: {((diff/cant_reg_inicial) * 100):.2f}%")
print(f"Cantidad de registros Final: {cant_reg_final}")

Cantidad de registros Inicial: 315305
% de reduccion de registros: 50.89%
Cantidad de registros Final: 154841


In [ ]:
df_spy.groupby('anio').size()

,0
anio,
2020,31605
2021,40002
2022,40174
2023,43060


In [ ]:
print(f"Cantidad de columnas: {len(df_spy.columns)}")

Cantidad de columnas: 31


In [ ]:
df_spy.columns

Index(['quote_date', 'expire_date', 'dte', 'underlying_last', 'strike',
       'strike_distance', 'strike_distance_pct', 'c_last', 'c_bid', 'c_ask',
       'c_iv', 'c_volume', 'c_delta', 'c_gamma', 'c_vega', 'c_theta',
       'daysToExp', 'c_mid_price', 'moneyness_category', 'anio', 'spread',
       'relative_spread', 'dte_year', 'c_volume_log', 'c_volume_log_scaled',
       'volume_zero', 'moneyness', 'iv_pct_rank', 'c_iv_min', 'iv_skew',
       'spread_mid_price'],
      dtype='object')

In [ ]:
df_spy.dtypes

,0
quote_date,datetime64[ns]
expire_date,datetime64[ns]
dte,float64
underlying_last,float64
strike,float64
strike_distance,float64
strike_distance_pct,float64
c_last,float64
c_bid,float64
c_ask,float64


#Generar CSV

In [ ]:
print(f"Filas: {df_spy.shape[0]}, Columns: {df_spy.shape[1]}")

Filas: 154841, Columns: 31


In [ ]:
output_name = 'spy_options_call_output_all.csv'
df_spy.to_csv(output_name, index=False)
print(f"se guardo el archivo CSV de todos los años")

se guardo el archivo CSV de todos los años


In [ ]:
# Finished!